In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Global imports
import os
from pytz import utc
from datetime import datetime

# Import vub Meetenet Vlaamse Banken API functions
from mvbc.config import Credentials
from mvbc.client import Base
from mvbc.objects import Catalog, Data
import mvbc.data_getter as dg

In [3]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

In [4]:
# This is only part of the development step, comment out if not developing!
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
mvbc_username = os.getenv('MEETNET_USERNAME') # Replace with your usernam
mvbc_password = os.getenv('MEETNET_PASSWORD') # Replace with your password
creds = Credentials(username=mvbc_username, password=mvbc_password)
b=Base(creds)
b.ping()

c = Catalog(credentials=creds)
df_unfiltered = c.data_points()

In [6]:
df_unfiltered

,Name,Description,PositionWKT,Location,Parameter,CurrentInterval,ParameterName,ParameterUnit,ParameterType
ID,,,,,,,,,
A2BGH1,A2 - Buoy,<b>Buoy: </b> Datawell - Directional waverider,POINT (3.11983333333333 51.3597222222222),A2B,GH1,30,10% highest waves,cm,1
A2BGHA,A2 - Buoy,<b>Buoy: </b> Datawell - Directional waverider,POINT (3.11983333333333 51.3597222222222),A2B,GHA,30,Wave height,cm,1
A2BGTZ,A2 - Buoy,<b>Buoy: </b> Datawell - Directional waverider,POINT (3.11983333333333 51.3597222222222),A2B,GTZ,30,Average wave period,s,1
A2BHLF,A2 - Buoy,<b>Buoy: </b> Datawell - Directional waverider,POINT (3.11983333333333 51.3597222222222),A2B,HLF,30,Height waves with period > 10 s,cm,1
A2BRHF,A2 - Buoy,<b>Buoy: </b> Datawell - Directional waverider,POINT (3.11983333333333 51.3597222222222),A2B,RHF,30,High frequent wave direction,deg,1
...,...,...,...,...,...,...,...,...,...
ZWNTLU,Zwin - Weather station,<b>Relative humidity:</b> Vaisala - HMP155 <br...,POINT (3.37166666666667 51.3402777777778),ZWN,TLU,10,Air temperature,°C,3
ZWNWC3,Zwin - Weather station,<b>Relative humidity:</b> Vaisala - HMP155 <br...,POINT (3.37166666666667 51.3402777777778),ZWN,WC3,10,Max 3-seconds wind gust (at 10 m height),m/s,4
ZWNWRS,Zwin - Weather station,<b>Relative humidity:</b> Vaisala - HMP155 <br...,POINT (3.37166666666667 51.3402777777778),ZWN,WRS,10,Average wind direction,deg,4


In [7]:
# Get the data for a certain weather station by name!
weather_station = 'Wandelaar'
dt_start = datetime(2022,9,30,tzinfo=utc) # timestamp with timezone
dt_end = datetime(2022,10,1,tzinfo=utc)
weather_station_data = \
    dg.get_data_by_weatherstation(
        weather_station,
        dt_start,
        dt_end,
        creds,
        df_unfiltered
    )

In [8]:
# Alternatively, if you you can get the data from the weather station closest to the location of interest
dt_start = datetime(2022,9,30,tzinfo=utc) # timestamp with timezone
dt_end = datetime(2022,10,1,tzinfo=utc)
location = 'bbc01'
location_of_interest = [2.81555595289144, 51.6918319416294]   # Replace this with location of interest

df_weather, weatherstation_information, all_wetaherstations = dg.get_longterm_weather_data(location_of_interest, dt_start, dt_end, df=df_unfiltered, credentials=creds)
df_weather = df_weather.resample('10T', axis=0).interpolate(method='linear', axis=0, limit=12)

In [10]:
df_weather.columns

Index(['mvbc_ThorntonbankSouthBuoy_10%_highest_waves',
       'mvbc_ThorntonbankSouthBuoy_Wave_height',
       'mvbc_ThorntonbankSouthBuoy_Average_wave_period',
       'mvbc_ThorntonbankSouthBuoy_Height_waves_with_period_>_10_s',
       'mvbc_ThorntonbankSouthBuoy_High_frequent_wave_direction',
       'mvbc_ThorntonbankSouthBuoy_Low_frequent_wave_direction',
       'mvbc_ThorntonbankSouthBuoy_Sea_water_temperature',
       'mvbc_WandelaarMeasuringpile_Tide_TAW',
       'mvbc_KwintebankBuoy_Significant_wave_height',
       'mvbc_WandelaarMeasuringpile_Air_pressure',
       'mvbc_OstendWeatherstation_Relative_humidity',
       'mvbc_WandelaarMeasuringpile_Air_temperature',
       'mvbc_OstendWeatherstation_Precipitation',
       'mvbc_OstendWeatherstation_Ground_temperature',
       'mvbc_ZeebruggeDaminstrumentationWeatherstation_Max_1-second_wind_gust_(at_10_m_height)',
       'mvbc_WandelaarMeasuringpile_Max_3-seconds_wind_gust_(at_10_m_height)',
       'mvbc_WandelaarMeasuringpile_Ave